In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, embedding_size):
        super(SelfAttention, self).__init__()
        self.embedding_size = embedding_size
        self.w_q = nn.Linear(embedding_size, embedding_size, bias = False)
        self.w_k = nn.Linear(embedding_size, embedding_size, bias = False)
        self.w_v = nn.Linear(embedding_size, embedding_size, bias = False)
        self.fc_out = nn.Linear(embedding_size, embedding_size, bias = False)
    
    def forward(self, query, key, value):
        query = self.w_q(query)
        key = self.w_k(key)
        value = self.w_v(value)

        attention_scores = torch.matmul(query, key.transpose(-1, -2)) * (self.embedding_size ** - 0.5)
        attention_scores = F.softmax(attention_scores, dim = -1)
        attention_out = torch.matmul(attention_scores, value)

        attention_out = self.fc_out(attention_out)
        return attention_out


In [4]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embedding_size, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embedding_size = embedding_size
        self.num_heads = num_heads
        self.head_dims = self.embedding_size // self.num_heads

        assert (self.num_heads * self.head_dims == self.embedding_size), \
            "embedding_size should be divisible by num_heads"
        
        self.w_q = nn.Linear(self.head_dims, self.head_dims, bias = False)
        self.w_k = nn.Linear(self.head_dims, self.head_dims, bias = False)
        self.w_v = nn.Linear(self.head_dims, self.head_dims, bias = False)
        self.fc_out = nn.Linear(self.head_dims, self.head_dims, bias = False)

    def forward(self, query, key, value):
        batch_size = query.shape[0]
        sequence_length_q, sequence_length_k, sequence_length_v = query.shape[1], key.shape[1], value.shape[1]

        query = query.reshape(batch_size, sequence_length_q, self.num_heads, self.head_dims)
        key = key.reshape(batch_size, sequence_length_k, self.num_heads, self.head_dims)
        value = value.reshape(batch_size, sequence_length_v, self.num_heads, self.head_dims)

        attention_scores = torch.matmul(query, key.transpose(-1, -2)) * (self.embedding_size ** -0.5)
        attention_scores = F.softmax(attention_scores, dim = -1)
        attention_out = torch.matmul(attention_scores, value)

        attention_out = attention_out.reshape(batch_size, sequence_length_q, self.num_heads*self.head_dims)
        return attention_out


In [6]:
batch_size, sequence_length, embedding_size = 1, 77, 768

query = torch.rand(batch_size, sequence_length, embedding_size)
key = torch.rand(batch_size, sequence_length, embedding_size)
value = torch.rand(batch_size, sequence_length, embedding_size)

self_attention = SelfAttention(embedding_size)
result = self_attention(query, key, value)
print(result.shape)

torch.Size([1, 77, 768])


In [5]:
batch_size, sequence_length, embedding_size = 1, 77, 768
num_head = 6
query = torch.rand([batch_size, sequence_length, embedding_size])
key = torch.rand([batch_size, sequence_length, embedding_size])
value = torch.rand([batch_size, sequence_length, embedding_size])
multi_head_self_attention = MultiHeadSelfAttention(embedding_size, num_head)
attention_out = multi_head_self_attention(query, key, value)

Q1: self-attention的公式中为什么除以根号dk

A：在self-attention的计算公式中,查询向量Q和键向量K的转置之间的相乘会生成一个注意力分数矩阵,这个矩阵中的每个元素表示序列中不同位置之间的相似性。
我们会先对这个矩阵进行softmax操作来生成注意力权重。但是,如果不对相乘的结果进行缩放的话,矩阵中元素的值太大或者太小的话,softmax得到的结果接近为0,
那也就是说可能会产生梯度消失或者梯度爆炸的问题,从而影响模型的训练。

Q：那么,为什么一定是除以根号dk呢？为什么不是除以dk呢？

A： 当我们进行相乘操作的时候,两个独立的标准正态分布的方差是dk。如果我们以dk为缩放系数的话,那么相乘结果的方差就会变为dk分之1。这会使得结果非常小,
甚至接近于0。特别是当dk很大的时候,这会导致softmax函数输出的注意力权重非常平均,不能突出重要的关系。
    但是,如果我们除以dk的话,softmax函数的输入就会变小,相乘的结果的方差就会变为1,从而使得softmax的输出结果更加稳定,梯度也比较适中,有利于反向传播
过程中的梯度更新。

Q2:multi-head self-attention的作用是什么？

A：1、使用了多头自注意力机制,可以丰富表达能力。
    首先,单头自注意力机制在每个时刻只能基于一个权重分布来计算注意力,这可能限制了模型捕捉不同类型依赖关系的能力。多头自注意力机制通过使用多个独立的注意力头,每个头都可以学习到不同的权重分布,从而捕捉不同位置之间的多种依赖关系。
    第二点,多头自注意力机制可以从多个角度,多个视角来理解输入序列,从而提高模型的表达能力。
    第三点,多头自注意力机制可以通过并行的方式来加速计算过程,提升了效率。